## 链的流式调用
*******
- 异步调用astream
- JSON流输出
- 使用流事件
- 注意：不是所有的组件都支持流式输出，当不支持的组件被封装在chain中后，最后的结果依旧可以使用流输出

In [1]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE"),
    )

### 异步调用astream
****

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | llm | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

||Why| don|'t| par|rots| use| cell| phones|?

|Because| they| already| have| their| own| "|tweet|-|er|"|!|||

### JSON流输出
****

In [5]:
from langchain_core.output_parsers import JsonOutputParser

# 创建一个链，将LLM的输出通过JsonOutputParser进行解析
# 注意：由于旧版本Langchain的一个bug，JsonOutputParser可能无法正确流式处理某些模型的结果
chain = (
    llm | JsonOutputParser()
)  

# 使用astream方法进行异步流式处理
# 发送提示要求模型以JSON格式输出法国、西班牙和日本的国家及其人口信息
async for text in chain.astream(
    "以JSON格式输出法国、西班牙和日本的国家列表及其人口数量。"
    '使用一个字典，包含外层键名为"countries"的国家列表。'
    "每个国家应该有键名`name`和`population`"
):
    print(text, flush=True)  # 打印每个流式返回的文本片段，并立即刷新输出缓冲区

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 670}]}
{'countries': [{'name': 'France', 'population': 670810}]}
{'countries': [{'name': 'France', 'population': 67081000}]}
{'countries': [{'name': 'France', 'population': 67081000}, {}]}
{'countries': [{'name': 'France', 'population': 67081000}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 67081000}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 67081000}, {'name': 'Spain', 'population': 469}]}
{'countries': [{'name': 'France', 'population': 67081000}, {'name': 'Spain', 'population': 469400}]}
{'countries': [{'name': 'France', 'population': 67081000}, {'name': 'Spain', 'population': 46940000}]}
{'countries': [{'name': 'France', 'population': 67081000}, {'name': 'Spain', 'population': 46940000}, {}]}
{'countries': [{'name': 'France', 'population': 67081000}, {'name': 'Spain', 'population': 469

### 事件流
*******
- 这是一个测试事件
- 可以将流的过程进行分解，从而事件更细颗粒度的控制
- langchain-core >= 0.2
- 事件流的颗粒度：

![](events.png)

In [6]:
# 注意对于版本langchain-core<0.3.37，需要显式地指定事件流版本
events = []
async for event in llm.astream_events("hello",version="v2"):
    events.append(event)

查看前三个events

In [7]:
events[:3]

[{'event': 'on_chat_model_start',
  'data': {'input': 'hello'},
  'name': 'ChatOpenAI',
  'tags': [],
  'run_id': '5cf4fe00-8cbf-4bd2-a3b7-509078e5aaaf',
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0},
  'parent_ids': []},
 {'event': 'on_chat_model_stream',
  'run_id': '5cf4fe00-8cbf-4bd2-a3b7-509078e5aaaf',
  'name': 'ChatOpenAI',
  'tags': [],
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0},
  'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-5cf4fe00-8cbf-4bd2-a3b7-509078e5aaaf')},
  'parent_ids': []},
 {'event': 'on_chat_model_stream',
  'run_id': '5cf4fe00-8cbf-4bd2-a3b7-509078e5aaaf',
  'name': 'ChatOpenAI',
  'tags': [],
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0},
  'data': {'chunk': AIMessageChunk(content

#### 事件过滤 - 按name
***
按照运行时配置的name来过滤事件

In [8]:
chain = llm.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
    include_names=["my_parser"],version="v2"
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_parser_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'my_parser', 'tags': ['seq:step:2'], 'run_id': '696f5f97-d770-48da-91d5-54f9c9d5b1ef', 'metadata': {}, 'parent_ids': ['048cc5a3-e3d8-400d-ad88-5782d3930a5d']}
{'event': 'on_parser_stream', 'run_id': '696f5f97-d770-48da-91d5-54f9c9d5b1ef', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {}}, 'parent_ids': ['048cc5a3-e3d8-400d-ad88-5782d3930a5d']}
{'event': 'on_parser_stream', 'run_id': '696f5f97-d770-48da-91d5-54f9c9d5b1ef', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {'countries': []}}, 'parent_ids': ['048cc5a3-e3d8-400d-ad88-5782d3930a5d']}
{'event': 'on_parser_stream', 'run_id': '696f5f97-d770-48da-91d5-54f9c9d5b1ef', 'name': 'my_parse

#### 事件过滤 - 按tag
***
按照tag来过滤事件

In [9]:
chain = (llm | JsonOutputParser()).with_config({"tags": ["my_chain"]})

max_events = 0
async for event in chain.astream_events(
    'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`',
    include_tags=["my_chain"],version="v2"
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_chain_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'RunnableSequence', 'tags': ['my_chain'], 'run_id': '462eefd2-9b86-45ac-bc36-cfd7d1644eb0', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`', additional_kwargs={}, response_metadata={})]]}}, 'name': 'ChatOpenAI', 'tags': ['seq:step:1', 'my_chain'], 'run_id': '74bad696-af1f-4c97-848d-feacaf1dd1e1', 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4', 'ls_model_type': 'chat', 'ls_temperature': 0.

#### 事件阶段过滤
***

In [10]:
num_events = 0

async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",version="v2"
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"Parser chunk: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # Truncate the output
        print("...")
        break

Chat model chunk: ''
Chat model chunk: ''
Chat model chunk: '{\n'
Parser chunk: {}
Chat model chunk: ' '
Chat model chunk: ' "'
Chat model chunk: 'countries'
Chat model chunk: '":'
Chat model chunk: ' [\n'
Parser chunk: {'countries': []}
Chat model chunk: '   '
Chat model chunk: ' {\n'
Parser chunk: {'countries': [{}]}
Chat model chunk: '     '
Chat model chunk: ' "'
Chat model chunk: 'name'
Chat model chunk: '":'
Chat model chunk: ' "'
Parser chunk: {'countries': [{'name': ''}]}
Chat model chunk: 'France'
Parser chunk: {'countries': [{'name': 'France'}]}
Chat model chunk: '",\n'
Chat model chunk: '     '
...
